In [13]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

FileNotFoundError: [Errno 2] No such file or directory: './content'

In [ ]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.preprocessing import MaxAbsScaler

def peaks(grid):
    x = np.linspace(-2, 2, grid)
    y = np.linspace(-2, 2, grid)
    x, y = np.meshgrid(x, y)
    z = 3*(1-x)**2 * np.exp(-(x**2) - (y+1)**2) - 10*(x/5 - x**3 - y**5) * np.exp(-x**2 - y**2) - 1/3 * np.exp(-(x+1)**2 - y**2)
    return x, y, z

def create_dataframe(grid):
   x, y, z = peaks(grid)
   data = {'X': x.flatten(), 'Y': y.flatten(), 'Z': z.flatten()}
   df = pd.DataFrame(data)
   return df

In [ ]:
df_25 = create_dataframe(grid=5)
df_1000 = create_dataframe(grid=32)

In [ ]:
def split_df(df):
    # df = df.sample(frac=1).reset_index(drop=True)  # Randomiza as linhas
    _input = np.vstack([df['X'], df['Y']]).T
    _output = np.array(df['Z'])
    return (_input, _output)

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import levenberg_marquardt as lm
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error
from keras import regularizers
from keras import initializers

class TrainWithSmallDataset:
    def __init__(self, batch_size=1000, store=25):
        self.batch_size = batch_size
        self.betters = []
        self.store = store

    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)


    def split_dataset(self, input, output, sup_input, sup_output):

      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.sup_dataset, self.sup_input, self.sup_output = self.create_dataset(sup_input, sup_output)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)

    def create_model(self, input_size=2, sumary=False):
      regularizer = regularizers.L2(0.2)
      initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

      self.model = tf.keras.Sequential([
          tf.keras.layers.Dense(30,
                                activation='tanh',
                                input_shape=(input_size,),
                                kernel_regularizer=regularizer,
                                kernel_initializer=initializer,
                                ),
          tf.keras.layers.Dense(16,
                                activation='tanh',
                                kernel_regularizer=regularizer,
                                kernel_initializer=initializer,
                                ),
          tf.keras.layers.Dense(1,
                                activation='linear',
                                kernel_regularizer=regularizer,
                                kernel_initializer=initializer,
        )])

      if (sumary == True):
        self.model.summary()

      self.model.compile(
          optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
          loss=tf.keras.losses.MeanSquaredError())

      self.lm_model = lm.ModelWrapper(
          tf.keras.models.clone_model(self.model))

      self.lm_model.compile(
          optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
          loss=lm.MeanSquaredError())

    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.sup_prediction = self.lm_model.predict(self.sup_input)
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()
          sup_pred = self.sup_prediction.flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)
          r2_sup = r2_score(self.sup_output, sup_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)
          mse_sup = mean_squared_error(self.sup_output, sup_pred)

          metrics = {
                          'r2': r2,
                          'r2_sup': r2_sup,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_sup': mse_sup,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics


    def plot_results(self):
      fig = plt.figure(figsize=(18, 6))
      ax0 = fig.add_subplot(1, 3, 1, projection='3d')
      ax2 = fig.add_subplot(1, 3, 2, projection='3d')
      ax1 = fig.add_subplot(1, 3, 3)

      ax0.set_title('Superficie Fit')
      ax0.scatter(self.sup_input[:, 0], self.sup_input[:, 1], self.sup_prediction, c='r', marker='s', label="lm")
      ax0.scatter(self.sup_input[:, 0], self.sup_input[:, 1], self.sup_output, c='b', marker='o', label="reference")
      ax0.legend()

      ax2.set_title('Teste Fit')
      ax2.scatter(self.test_input[:, 0], self.test_input[:, 1], self.test_prediction, c='r', marker='s', label="lm")
      ax2.scatter(self.test_input[:, 0], self.test_input[:, 1], self.test_output, c='b', marker='o', label="reference")
      ax2.legend()

      ax1.plot(self.results.history['loss'], label='Erro Treino')
      ax1.plot(self.results.history['val_loss'], label='Erro validação')
      ax1.set_title('Histórico de Treinamento')
      ax1.set_ylabel('Função de Custo')
      ax1.set_xlabel('Épocas de Treinamento')
      ax1.legend(['Erro Treino', 'Erro validação'])

      return(fig)

2024-01-30 22:46:46.261626: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-30 22:46:46.294788: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 22:46:46.294836: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 22:46:46.296032: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-30 22:46:46.302295: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-30 22:46:46.302827: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [25]:
from keras import saving
from keras import models
import pickle

class Tester:
  def __init__(self, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input_25, self.output_25 = split_df(df_25)
    self.input_1000, self.output_1000 = split_df(df_1000)

  def PrepareTraining(self):
    trainer = TrainWithSmallDataset()
    trainer.split_dataset(self.input_25, self.output_25, self.input_1000, self.output_1000)
    return (trainer)

  def Train(self, trainer, epochs=1000):
    trainer.create_model()
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path)
  
  def SaveFigs(self, trainer, fileName):
    path = f"../results/{fileName}.png" 
    open(path,'w').close()
    figure = trainer.plot_results()
    figure.savefig(path)
    plt.close(figure)

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)
  
  def LoadDataset(self, fileName, trainer=None):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'rb') as f:
      _train, _vt, _val, _test = pickle.load(f)
    trainer._train, trainer._vt, trainer._val, trainer._test = _train, _vt, _val, _test

  def LoopWeights(self, sort_by, model_boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ Dataset [{idx + 1}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] <= model_boundarie): # should be >= to acsending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.SaveFigs(trainer, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

  # brief: for each dataset
  def LoopDataset(self, sort_by, model_boundarie):
    for j in range(self.dataset_run_times):
      trainer = self.PrepareTraining() # split dataset
      if (self.LoopWeights(sort_by, model_boundarie, trainer, j) == True):
        self.SaveDataset(trainer, f"dataset_{j}")
        self.DisplayBetterResults('mse_sup', j)
      self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    excel_path = f'../results/metrics_{dataset}.xlsx'
    df.to_excel(excel_path, index=True)
    print(f"DataFrame salvo em {excel_path}")

# Treinando com 25 dados


In [26]:
tester = Tester(run_times=100, dataset_run_times=50)

In [27]:
tester.LoopDataset(sort_by='mse_sup', model_boundarie=2.5)

+++++++++++ Dataset [1] | 1 ++++++++++++++++++
Elapsed time:  31.239655102999677
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ Dataset [1] | 2 ++++++++++++++++++
Elapsed time:  29.574268076000408
Stopped at epoch:  888
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [1] | 3 ++++++++++++++++++
Elapsed time:  31.432626757000435
Stopped at epoch:  969
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [1] | 4 ++++++++++++++++++
Elapsed time:  3.8927250560000175
Stopped at epoch:  108
1/1 [==============================] - 0s 15ms/step
+++++++++++ Dataset [1] | 5 ++++++++++++++++++
Elapsed time:  30.25205837700014
Stopped at epoch:  955
1/1 [==============================] - 0s 17ms/step
+++++++++++ Dataset [1] | 6 ++++++++++++++++++
Elapsed time:  6.077559436000229
Stopped at epoch:  169
1/1 [==============================] - 0s 19ms/step
+++++++++++ Dataset [1] | 7 ++++++++++++++++++
Elapsed time:  5.5176

2024-01-30 23:54:01.125511: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  33.82731232500009
Stopped at epoch:  0
1/1 [==============================] - 0s 18ms/step
+++++++++++ Dataset [1] | 82 ++++++++++++++++++
Elapsed time:  2.7624177830002736
Stopped at epoch:  82
1/1 [==============================] - 0s 17ms/step
+++++++++++ Dataset [1] | 83 ++++++++++++++++++
Elapsed time:  5.235062968999955
Stopped at epoch:  140
1/1 [==============================] - 0s 18ms/step
+++++++++++ Dataset [1] | 84 ++++++++++++++++++
Elapsed time:  0.7748843079998551
Stopped at epoch:  9
1/1 [==============================] - 0s 19ms/step
+++++++++++ Dataset [1] | 85 ++++++++++++++++++
Elapsed time:  32.704114574000414
Stopped at epoch:  0
1/1 [==============================] - 0s 18ms/step
+++++++++++ Dataset [1] | 86 ++++++++++++++++++
Elapsed time:  9.199845797999842
Stopped at epoch:  269
1/1 [==============================] - 0s 23ms/step
+++++++++++ Dataset [1] | 87 ++++++++++++++++++
Elapsed time:  6.871192792999864
Stopped at epoch:  195
1/1 [=======

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_0_1,0.819936,0.642792,0.117014,0.749868,0.727766,0.755691,2.388649,0.523147,4.048838,2.285993
model_0_5,0.795389,0.641933,0.266818,0.695274,0.680432,0.858708,2.394393,0.434392,4.932535,2.683464
model_0_0,0.811915,0.638489,0.665588,0.707458,0.706243,0.789352,2.417423,0.198131,4.735322,2.466727
model_0_2,0.770011,0.628307,-0.421876,0.679358,0.640796,0.965214,2.485514,0.842427,5.190163,3.016294
model_0_4,0.788534,0.627877,0.707757,0.668027,0.669726,0.887477,2.488388,0.173147,5.373582,2.773365
model_0_3,0.790835,0.627727,0.680671,0.672747,0.673319,0.877820,2.489388,0.189195,5.297183,2.743189


DataFrame salvo em ../results/metrics_0.xlsx
+++++++++++ Dataset [2] | 1 ++++++++++++++++++
Elapsed time:  2.949406968999938
Stopped at epoch:  80
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [2] | 2 ++++++++++++++++++
Elapsed time:  1.6974577909995787
Stopped at epoch:  38
1/1 [==============================] - 0s 20ms/step
+++++++++++ Dataset [2] | 3 ++++++++++++++++++
Elapsed time:  1.9329621550004958
Stopped at epoch:  44
1/1 [==============================] - 0s 13ms/step
+++++++++++ Dataset [2] | 4 ++++++++++++++++++
Elapsed time:  1.789421716999641
Stopped at epoch:  38
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [2] | 5 ++++++++++++++++++
Elapsed time:  3.1770550119999825
Stopped at epoch:  81
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [2] | 6 ++++++++++++++++++
Elapsed time:  0.6122916260001148
Stopped at epoch:  8
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [2] | 7 

...

DataFrame salvo em ../results/metrics_2.xlsx
+++++++++++ Dataset [4] | 1 ++++++++++++++++++
Elapsed time:  0.5398561699994389
Stopped at epoch:  7
1/1 [==============================] - 0s 16ms/step
+++++++++++ Dataset [4] | 2 ++++++++++++++++++
Elapsed time:  0.582248484000047
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ Dataset [4] | 3 ++++++++++++++++++
Elapsed time:  0.5252898720000303
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [4] | 4 ++++++++++++++++++
Elapsed time:  0.5721677850006017
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ Dataset [4] | 5 ++++++++++++++++++
Elapsed time:  0.4585245819998818
Stopped at epoch:  6
1/1 [==============================] - 0s 19ms/step
+++++++++++ Dataset [4] | 6 ++++++++++++++++++
Elapsed time:  0.5371757889997752
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ Dataset [4] | 7 ++++

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_5_10,0.916445,0.692804,0.304457,0.883456,0.781136,0.350661,2.054222,1.395376,0.786784,1.091080
model_5_0,0.870343,0.682548,0.257972,0.750225,0.681572,0.544143,2.122803,1.488632,1.686216,1.587425
model_5_22,0.868787,0.678412,0.051970,0.798460,0.672781,0.550673,2.150462,1.901908,1.360589,1.631248
model_5_21,0.867476,0.675750,-0.039947,0.794135,0.651358,0.556175,2.168258,2.086309,1.389782,1.738046
model_5_23,0.866561,0.668752,-0.039781,0.803072,0.657443,0.560016,2.215058,2.085976,1.329454,1.707715
model_5_13,0.867954,0.664613,0.174127,0.802238,0.699919,0.554167,2.242735,1.656840,1.335084,1.495961
model_5_7,0.892109,0.659920,0.289112,0.843017,0.750667,0.452797,2.274114,1.426160,1.059785,1.242973
model_5_25,0.846716,0.656651,-0.103216,0.797234,0.640726,0.643302,2.295979,2.213237,1.368863,1.791049
model_5_24,0.849336,0.653144,0.165098,0.751221,0.663559,0.632304,2.319426,1.674954,1.679491,1.677222
model_5_27,0.857442,0.652886,0.097345,0.758173,0.654634,0.598284,2.321156,1.810878,1.632558,1.721718


DataFrame salvo em ../results/metrics_5.xlsx
+++++++++++ Dataset [7] | 1 ++++++++++++++++++
Elapsed time:  2.798192160999861
Stopped at epoch:  71
1/1 [==============================] - 0s 17ms/step
+++++++++++ Dataset [7] | 2 ++++++++++++++++++
Elapsed time:  4.374659324999811
Stopped at epoch:  123
1/1 [==============================] - 0s 18ms/step
+++++++++++ Dataset [7] | 3 ++++++++++++++++++
Elapsed time:  3.7473653089991785
Stopped at epoch:  82
1/1 [==============================] - 0s 19ms/step
+++++++++++ Dataset [7] | 4 ++++++++++++++++++
Elapsed time:  12.000616760999947
Stopped at epoch:  250
1/1 [==============================] - 0s 16ms/step
+++++++++++ Dataset [7] | 5 ++++++++++++++++++
Elapsed time:  0.5347364279996327
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [7] | 6 ++++++++++++++++++
Elapsed time:  0.5496298210000532
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ Dataset [7] | 7

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_6_4,0.850255,0.723350,-0.109181,0.468170,0.646440,0.628447,1.849961,0.863166,2.766424,1.814795
model_6_0,0.866644,0.713799,-0.221547,0.580273,0.694725,0.559667,1.913828,0.950610,2.183298,1.566954
model_6_1,0.847765,0.666597,-0.555967,0.654875,0.707174,0.638896,2.229467,1.210855,1.795243,1.503049
model_6_2,0.820833,0.664328,-0.545273,0.537853,0.648690,0.751926,2.244639,1.202533,2.403955,1.803244
model_6_3,0.792993,0.629895,-0.602046,0.476087,0.613090,0.868764,2.474891,1.246714,2.725244,1.985979


DataFrame salvo em ../results/metrics_6.xlsx
+++++++++++ Dataset [8] | 1 ++++++++++++++++++
Elapsed time:  0.5147894880001331
Stopped at epoch:  9
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [8] | 2 ++++++++++++++++++
Elapsed time:  0.47903315399980784
Stopped at epoch:  7
1/1 [==============================] - 0s 13ms/step
+++++++++++ Dataset [8] | 3 ++++++++++++++++++
Elapsed time:  0.45530430799954047
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [8] | 4 ++++++++++++++++++
Elapsed time:  0.4831036630002927
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [8] | 5 ++++++++++++++++++
Elapsed time:  0.6596904980006002
Stopped at epoch:  14
1/1 [==============================] - 0s 13ms/step
+++++++++++ Dataset [8] | 6 ++++++++++++++++++
Elapsed time:  0.5045406399995045
Stopped at epoch:  8
1/1 [==============================] - 0s 14ms/step
+++++++++++ Dataset [8] | 7 